I - DATA

In [8]:
# Taux US
from pathlib import Path
import os
import pandas as pd

try:
    from fredapi import Fred
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "fredapi"])
    from fredapi import Fred

from skfin.dataloaders.cache import CacheManager

# ----------------------------
# 0) Réglages
# ----------------------------
CACHE_DIR = Path("") / "data"            # unique dossier de cache
FILENAME  = "rates_daily.parquet"           # unique nom de fichier
SERIES_IDS = [
    "EFFR","DFEDTARU","DFEDTARL","IORB","RRPONTTLD",
    "DGS1","DGS2","DGS5","DGS7","DGS10","DGS30",
    "T10Y2Y","T10Y3M","T5YIE","T10YIE","TEDRATE","BAMLH0A0HYM2",
    "WALCL","M2SL","CPIAUCSL","UNRATE"
]
# API key :
FRED_API_KEY = os.getenv("FRED_API_KEY", "2d98c6caa7753b549869a87c5636fea0 ").strip()

cm = CacheManager(cache_dir=CACHE_DIR)

# ----------------------------
# 1) Source loader
# ----------------------------
def load_rates_from_fred() -> pd.DataFrame:
    fred = Fred(api_key=FRED_API_KEY)
    # téléchargement en dict de Series → concat colonne
    df = pd.concat({sid: fred.get_series(sid) for sid in SERIES_IDS}, axis=1)
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    # subset + features
    cols = ["EFFR","IORB","RRPONTTLD","DGS2","DGS10","T10Y2Y","T10Y3M","T5YIE","TEDRATE"]
    rates_daily = df[cols].dropna()

    # cible à 5 jours (forward sur 10Y)
    rates_daily["dDGS10_5d"] = rates_daily["DGS10"].shift(-5) - rates_daily["DGS10"]

    # lags simples
    for col in cols:
        for L in (1, 2, 5):
            rates_daily[f"{col}_lag{L}"] = rates_daily[col].shift(L)

    # nettoyage final
    rates_daily = rates_daily.dropna().copy()

    # encodage de fréquence
    rates_daily = rates_daily.asfreq("D")  # reste NaN les jours non ouvrés

    return rates_daily

cache_path = cm.cache_dir / FILENAME
rates_daily = cm.get_cached_dataframe(
    filename=FILENAME,
    loader_func=load_rates_from_fred,  # <-- c’est ici que la fonction est appelée si pas de cache
    force_reload=False,                # True = re-télécharge depuis FRED
)
